In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os,time
import glob
%matplotlib inline

In [ ]:
"""
The Camera class is used to calculate calibration parameters
based on a set of calibration images passed to the calibrate()
method. This class can be used dewarp an input image passed
as an np blob or as a file name, with the undistort() method. 
"""
class Camera():
    def __init__(self, verbose=False):
        self.mtx = None
        self.rvecs = None
        self.tvecs = None
        self.shape = None
        self.dist = None
        self.verbose = verbose
        
    def calibrate(self, calibration_folder):
        images = glob.glob("%s/calibration*.jpg" % calibration_folder)
        print('performing calibration on %d images in %s...' % (len(images), calibration_folder))
        
        objpoints = []
        imgpoints = []
        nx = 9
        ny = 6
        objp = np.zeros((ny*nx, 3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

        for i in images:
            if (self.verbose):
                print("reading calibration file: %s" % i)
            raw_img = mpimg.imread(i)
            gray_img = cv2.cvtColor(raw_img, cv2.COLOR_RGB2GRAY)
            ret, corners = cv2.findChessboardCorners(gray_img, (nx, ny), None)
            self.shape = gray_img.shape[::-1]
            if (ret):
                imgpoints.append(corners)
                objpoints.append(objp)
                
            elif (self.verbose):
                print("WARNING:: wrong # of corners found in : ", i)

        ret, self.mtx, self.dist, self.rvecs, self.tvecs = cv2.calibrateCamera(objpoints,
                                                                          imgpoints, self.shape, None, None)
        print("calibration complete")
        if (self.verbose):
            print("mtx: ", self.mtx)
            print("dist:", self.dist)
            print("rvecs: ", self.rvecs)
            print("tvecs: ", self.tvecs)
                
    def test_calibration(self, img_file):
        img = cv2.imread(img_file)
        dst = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        plt.imshow(dst)
    
    def undistort(self, img):
        if (isinstance(img, np.ndarray)):
            return cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        elif (isinstance(img, str)):
            raw = cv2.imread(img)
            return cv2.undistort(raw, self.mtx, self.dist, None, self.mtx)
        else:
            raise TypeError("input argument is invalid type", type(img))


camera = Camera(verbose=False)
camera.calibrate("camera_cal")
plt.imshow(cv2.imread("camera_cal/calibration1.jpg"))

In [ ]:
"""
The SobelFilter class is used process a raw image
and return a processed binary image to extract 
the lane lines
"""
class SobelFilter():
    def __init__(self, verbose=False):
        self.verbose=verbose
        self.max = 1

    def grayscale(self, img):
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    def hls_select(self, img, thresh=(145, 255)):
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        s_channel = hls[:,:,2]

        binary = np.zeros_like(img)
        binary[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = self.max
        
        return binary
        
    def magnitude_threshold(self, img, sobel_kernel=5, threshold=(40, 100)):
        gray = self.grayscale(img)
        x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        magnitude = np.sqrt(x*x + y*y)
        scaled = np.uint8(255* magnitude / np.max(magnitude))
        binary = np.zeros_like(img)
        binary[(scaled >= threshold[0]) & (scaled <= threshold[1])] = self.max
        
        return binary
    
    def absolute_threshold(self, img, orient='x', threshold=(40, 100)):
        gray = self.grayscale(img)
        if orient == 'x':
            sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
        if orient == 'y':
            sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
        scaled = np.uint8(255*sobel / np.max(sobel))
        binary = np.zeros_like(img)
        binary[(scaled >= threshold[0]) & (scaled <= threshold[1])] = self.max
        
        return binary
        
    def direction_threshold(self, img, sobel_kernel=15, thresh=(0.7, 1.3)):
        gray = self.grayscale(img)
        x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
        y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
        direction = np.arctan2(np.absolute(y), np.absolute(x))
        binary = np.zeros_like(img)
        binary[(direction >= thresh[0]) & (direction <= thresh[1])] = self.max        

        return binary
    
    def process(self, img):
        x = self.absolute_threshold(img, 'x')
        y = self.absolute_threshold(img, 'y')
        magnitude = self.magnitude_threshold(img)
        direction = self.direction_threshold(img)        
        s = self.hls_select(img)
        
        output = np.zeros_like(img)
        output[((x == self.max) & (y == self.max)) |
               ((magnitude == self.max) & (direction == self.max)) |
               (s == self.max)] = self.max
        
        return output

In [ ]:
"""
The LaneLine class is a simple low pass filter around a 2nd order polynomial 
"""
class LaneLine():
    def __init__(self):
        self.coeff = [0, 0, 0]
        self.initialized = False
        self.curvature = 0
    
    def update_coeff(self, measurements):
        # update the filtered coefficients with new measurements
        if (self.initialized == True):
            self.coeff[0] = self.coeff[0] * 0.9 + measurements[0] * 0.1
            self.coeff[1] = self.coeff[1] * 0.9 + measurements[1] * 0.1
            self.coeff[2] = self.coeff[2] * 0.9 + measurements[2] * 0.1
        else:
            self.coeff[0] = measurements[0]
            self.coeff[1] = measurements[1]
            self.coeff[2] = measurements[2]
            self.initialized = True

    def update_curvature(self, curvature):
        self.curvature = curvature
            
    def get_curvature(self):
        return self.curvature

    def get_coeff(self):
        return self.coeff

    def reset(self):
        self.initialized = False


"""
The LaneDetector class will detect lanes from images performing the following:
 1) Applies a distortion correction to raw images.
 2) Uses a set of Sobel filters to threshold the relevant lane line pixels
 3) Applies a perspective transform to rectify binary image ("birds-eye view").
 4) Detects lane pixels and fit to find the lane boundary.
 5) Determines the curvature of the lane and vehicle position with respect to center.
 6) Warps the detected lane boundaries back onto the original image.
 7) Generates a final image of the lane boundaries and numerical estimation of lane curvature and vehicle position.
"""        
class LaneDetector():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.sobel = SobelFilter()
        self.M_BEV = None
        self.M_IMG = None
        self.left_line = LaneLine()
        self.right_line = LaneLine()
        self.offset = 0
        self.curvature = 0
        
    def reset(self):
        self.M_BEV = None
        self.M_IMG = None
        self.left_line.reset()
        self.right_line.reset()
        self.offset = 0
        
    def print_debug(self):
        print("Left Line Curvature (m): ", self.left_line.get_curvature())
        print("Right Line Curvature (m): ", self.right_line.get_curvature())
        print("Average Curvature (m): ", self.curvature)
        print("Offset (m): ", self.offset)
        
    def threshold(self, img):        
        return self.sobel.process(img)
    
    def transform_to_bev(self, img):
        img_size = img.shape[::-1][-2:]

        if (self.M_BEV == None):
            src = np.float32([[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
                              [((img_size[0] / 6) - 10), img_size[1]],
                              [(img_size[0] * 5 / 6) + 60, img_size[1]],
                              [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
            dst = np.float32([[(img_size[0] / 4), 0],
                              [(img_size[0] / 4), img_size[1]],
                              [(img_size[0] * 3 / 4), img_size[1]],
                              [(img_size[0] * 3 / 4), 0]])

            self.M_BEV = cv2.getPerspectiveTransform(src, dst)
        
        return cv2.warpPerspective(img, self.M_BEV, img_size)

    def transform_to_image(self, img):
        img_size = img.shape[::-1][-2:]
        
        if (self.M_IMG == None):
            dst = np.float32([[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
                              [((img_size[0] / 6) - 10), img_size[1]],
                              [(img_size[0] * 5 / 6) + 60, img_size[1]],
                              [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
            src = np.float32([[(img_size[0] / 4), 0],
                              [(img_size[0] / 4), img_size[1]],
                              [(img_size[0] * 3 / 4), img_size[1]],
                              [(img_size[0] * 3 / 4), 0]])

            self.M_IMG = cv2.getPerspectiveTransform(src, dst)        
            
        return cv2.warpPerspective(img, self.M_IMG, img_size)
        
    def calculate_curvature(self, y_eval, midpoint):
        # meters per pixel in y
        ym_per_pix = 30 / 720 

        # meters per pixel in x
        xm_per_pix = 3.7 / 700 
        
        left_curv = ((1 + (2*self.left_line.get_coeff()[0]*y_eval*ym_per_pix + self.left_line.get_coeff()[1])**2)**1.5) / np.absolute(2*self.left_line.get_coeff()[0])
        right_curv = ((1 + (2*self.right_line.get_coeff()[0]*y_eval*ym_per_pix + self.right_line.get_coeff()[1])**2)**1.5) / np.absolute(2*self.right_line.get_coeff()[0])
        
        # find the offset of ego within the lane
        average = (self.left_line.get_coeff()[2] + self.right_line.get_coeff()[2]) / 2        
        self.offset = xm_per_pix * (midpoint - average)     
        
        # update internal members for curvature and offset
        self.left_line.update_curvature(left_curv)
        self.right_line.update_curvature(right_curv)
        self.curvature = (left_curv + right_curv) / 2.0
    
    def detect_lanes(self, img, produce_visuals=False):                
        # Create an output image to draw on and visualize the result
        out_img = img*255

        # Take a histogram of the bottom half of the image
        histogram = np.sum(img[img.shape[0]//2:,:], axis=0)

        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint], axis=0)[0]
        rightx_base = np.argmax(histogram[midpoint:], axis=0)[0] + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        
        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(img.shape[0]//nwindows)
        
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Current positions to be updated later for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        
        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = img.shape[0] - (window + 1) * window_height
            win_y_high = img.shape[0] - window * window_height
            
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
        
            # Draw the windows on the visualization image
            if (produce_visuals):
                cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0,255,0), 2) 
                cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0,255,0), 2) 
            
            # Identify the nonzero pixels in x and y within the window #
            good_left_inds = ((nonzeroy >= win_y_low) & 
                              (nonzeroy < win_y_high) &
                              (nonzerox >= win_xleft_low) &
                              (nonzerox < win_xleft_high)).nonzero()[0]
            
            good_right_inds = ((nonzeroy >= win_y_low) &
                               (nonzeroy < win_y_high) & 
                               (nonzerox >= win_xright_low) &
                               (nonzerox < win_xright_high)).nonzero()[0]
        
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
        
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        
        # Update the internal members which are filtered 
        # polynomial coeffcients
        self.left_line.update_coeff(np.polyfit(lefty, leftx, 2))
        self.right_line.update_coeff(np.polyfit(righty, rightx, 2))
                
        # To improve processing efficieny do not always generate
        # the intermediate warped image unless required
        if (produce_visuals):
            left_fit = self.left_line.get_coeff()
            right_fit = self.right_line.get_coeff()
        
            # Creates a vector in y to generate the polynomials frome
            ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )

            # Generate the left and right poly lines (x)
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

            # Color in the left and right lane regions
            out_img[lefty, leftx] = [255, 0, 0]
            out_img[righty, rightx] = [0, 0, 255]

            # Plots the left and right polynomials on the lane lines        
            if (left_fitx.all() > 0 and left_fitx.all() < len(out_img[1])):
                out_img[np.int_(ploty), np.int_(left_fitx)] = [0, 255, 255]
            if (right_fitx.all() > 0 and right_fitx.all() < len(out_img[1])):
                out_img[np.int_(ploty), np.int_(right_fitx)] = [0, 255, 255]
        
            return out_img
        else:
            return None
    
    def generate_lane_image(self, img):
        out_img = np.zeros_like(img)
        
        # generate an image in BEV of just the polynomials
        left = self.left_line.get_coeff()
        right = self.right_line.get_coeff()
        
        nonzero = np.ones_like(img).nonzero()
        y = np.array(nonzero[0])
        x = np.array(nonzero[1])
        
        # draw in the lane region by filling all pixels which fall between
        # the left and right polynomial
        lane_inds = ((x > (left[0]*(y**2) + left[1]*y + left[2])) &
                     (x < (right[0]*(y**2) + right[1]*y + right[2])))
        out_img[y[lane_inds], x[lane_inds]] = [0, 255, 255]

        # draw the left and right lane lines by filling in "thickness" pixels
        # on either side of the polynomial line (this is very inefficient)
        thickness = 20
        left_line_inds = ((x > (left[0]*(y**2) + left[1]*y + left[2] - thickness)) &
                          (x < (left[0]*(y**2) + left[1]*y + left[2] + thickness)))
        out_img[y[left_line_inds], x[left_line_inds]] = [255, 0, 0]
        
        right_line_inds = ((x > (right[0]*(y**2) + right[1]*y + right[2] - thickness)) &
                           (x < (right[0]*(y**2) + right[1]*y + right[2] + thickness)))
        out_img[y[right_line_inds], x[right_line_inds]] = [0, 0, 255]        

        # transform the bird's eye view image back into image space
        transformed = self.transform_to_image(out_img)
        
        # overlay the transformed image onto the original image
        combined = cv2.addWeighted(img, 1.0, transformed, 0.5, 0)
        
        # text parameters
        font = cv2.FONT_HERSHEY_SIMPLEX
        color = (255, 0, 0)
        thickness = 2
   
        # add text to the image to show the curvature and offset
        final = cv2.putText(combined, 'Lane Curvature = %.1f (m)' % self.curvature,
                            (50, 50), font, 1, color, thickness, cv2.LINE_AA)
        final = cv2.putText(combined, 'Offset = %.2f (m)' % self.offset,
                            (50, 100), font, 1, color, thickness, cv2.LINE_AA)

        return final
    
    def process(self, img):
        start = time.time()
        thresholded = self.threshold(img)
        #print ("threshold : %.3f" % (time.time() - start))

        start = time.time()
        transformed = self.transform_to_bev(thresholded)
        #print ("transform_to_bev : %.3f" % (time.time() - start))
        
        start = time.time()
        lane_detected = self.detect_lanes(transformed)
        #print ("detect_lanes : %.3f" % (time.time() - start))
        
        start = time.time()
        self.calculate_curvature(img.shape[0], img.shape[1]/2)
        #print ("calculate_curvature : %.3f" % (time.time() - start))
        
        start = time.time()
        final = self.generate_lane_image(img)
        #print ("generate_lane_image : %.3f" % (time.time() - start))
        
        if (self.verbose):
            self.print_deubg()
        
        return final

lane_detector = LaneDetector(verbose=False)

In [ ]:
def save(image, name, directory):
    cv2.imwrite(directory + "/" + i, image)

test_images = os.listdir("test_images/")

for i in test_images:
    if (i[-3:] == 'jpg'):
        img = mpimg.imread('test_images/%s' % i)
        undistored_image = camera.undistort(img)
        output = lane_detector.process(undistored_image)
        save(output, i, "output_images")
        lane_detector.reset()
        print("procesed : %s" % i)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # Copy the image argument frame locally, 
    # otherwise, the full video will be a single frame
    img = image

    # Pass the raw image through the camera claass
    # to find the undistored version
    undistored_image = camera.undistort(img)
    
    # Pass the undistored image through the lane
    # detector to find the post processed version
    output = lane_detector.process(undistored_image)

    # Returns the process imaged back to the 
    # video processor
    return output

In [ ]:
output_video = 'output_images/project_video_output.mp4'
input_video = VideoFileClip('project_video.mp4')
clip = input_video.fl_image(process_image)
%time clip.write_videofile(output_video, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video))